# PDF Document Loaders in LangChain

이 노트북에서는 LangChain에서 제공하는 다양한 PDF 문서 로더들의 특징과 사용법을 학습합니다.

## 목차
1. [PDF 개요](#intro)
2. [환경 설정](#setup)
3. [PyPDFLoader](#pypdf)
4. [PyPDFLoader with OCR](#pypdf-ocr)
5. [PyMuPDFLoader](#pymupdf)
6. [UnstructuredPDFLoader](#unstructured)
7. [PyPDFium2Loader](#pypdfium2)
8. [로더 비교 및 선택 가이드](#comparison)
9. [핵심 요약](#summary)

## 1. PDF 개요 <a id='intro'></a>

[Portable Document Format (PDF)](https://en.wikipedia.org/wiki/PDF)는 ISO 32000으로 표준화된 파일 형식으로, Adobe가 1992년에 문서를 제시하기 위해 개발했습니다. 이는 응용 소프트웨어, 하드웨어 및 운영 시스템에 독립적인 방식으로 텍스트 서식 및 이미지를 포함합니다.

### LangChain의 PDF 로더들

LangChain은 다양한 PDF 파서와 통합됩니다:
- **간단하고 저수준**: 기본적인 텍스트 추출
- **고급 기능**: OCR, 이미지 처리, 문서 레이아웃 분석

올바른 선택은 사용자의 애플리케이션에 따라 달라집니다.

### 참고 자료
- [LangChain 공식 문서 - PDF Loaders](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/)

## 2. 환경 설정 <a id='setup'></a>

필요한 라이브러리를 설치하고 환경을 설정합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# 테스트용 파일 경로 설정
FILE_PATH = "../ADsP.pdf"  # 실제 파일 경로로 변경 필요

In [3]:
# 메타데이터를 보기 좋게 출력하는 헬퍼 함수
def show_metadata(docs):
    """문서의 메타데이터를 보기 좋게 출력하는 함수"""
    if docs:
        print("[metadata keys]")
        print(list(docs[0].metadata.keys()))
        print("\n[metadata examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

## 3. PyPDFLoader <a id='pypdf'></a>

`PyPDFLoader`는 가장 기본적인 PDF 로더입니다. 간단하고 빠르지만 복잡한 레이아웃이나 이미지 처리에는 제한이 있습니다.

### 특징
- 간단하고 빠른 텍스트 추출
- 페이지별로 문서 분할
- 기본적인 메타데이터 제공

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# PyPDF 로더 초기화
loader = PyPDFLoader(FILE_PATH)

# PDF 문서 로드
docs = loader.load()

print(f"로드된 문서 수: {len(docs)}")
print(f"\n첫 번째 페이지 내용 (처음 500자):")
print(docs[0].page_content[:500])

로드된 문서 수: 1

첫 번째 페이지 내용 (처음 500자):
<ADsP 요약정리 및 오답노트>-1과목-(객관식)데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성Cinematch -> 넷플릭스에서 개발한 알고리즘데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임트레이딩, 공급, 수요예측 -> 에너지 산업CRM -> 고객관계관리 데이터베이스 (기업내부)ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발                                          2) 새로운 가치 창출                                          3) 분석 기술 발전cf. 전문인력증가는 가치측정과 관련 없음사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화유형분석 -> 특성에따라 분류할때 사용한다.핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가K-NN 분석기법 - 딥러닝과 관련 xLSTM, Autoencoder, RNN -> 딥러닝과 관련 oCaffe, Tnsorflow


In [5]:
# PyPDFLoader의 메타데이터 확인
show_metadata(docs)

[metadata keys]
['producer', 'creator', 'creationdate', 'source', 'total_pages', 'page', 'page_label']

[metadata examples]
producer     : PyPDF2
creator      : PyPDF
creationdate : 
source       : ../ADsP.pdf
total_pages  : 1
page         : 0
page_label   : 1


## 4. PyPDFLoader with OCR <a id='pypdf-ocr'></a>

일부 PDF에는 스캔된 문서나 그림 내에 텍스트 이미지가 포함되어 있습니다. `rapidocr-onnxruntime` 패키지를 사용하여 이미지에서 텍스트를 추출할 수 있습니다.

### 설치
```bash
pip install -qU rapidocr-onnxruntime
```

### 특징
- 이미지에서 텍스트 추출 (OCR)
- 스캔된 문서 처리 가능
- `extract_images=True` 옵션 필요

In [6]:
# OCR 기능을 활성화한 PyPDF 로더
# 논문 PDF를 예시로 사용 (이미지가 포함된 문서)
ocr_loader = PyPDFLoader(
    "https://arxiv.org/pdf/2103.15348.pdf", 
    extract_images=True  # OCR 활성화
)

# PDF 페이지 로드
ocr_docs = ocr_loader.load()

# 이미지가 포함된 페이지 내용 확인
print("5번째 페이지 내용 (처음 300자):")
print(ocr_docs[4].page_content[:300])
print("\nOCR이 이미지에서 텍스트를 추출했음을 확인할 수 있습니다.")

5번째 페이지 내용 (처음 300자):
LayoutParser: A Unified Toolkit for DL-Based DIA 5
Table 1: Current layout detection models in the LayoutParser model zoo
Dataset Base Model1 Large ModelNotes
PubLayNet [38] F / M M Layouts of modern scientific documents
PRImA [3] M - Layouts of scanned modern magazines and scientific reports
Newspaper

OCR이 이미지에서 텍스트를 추출했음을 확인할 수 있습니다.


In [7]:
# OCR 로더의 메타데이터 확인
show_metadata(ocr_docs)

[metadata keys]
['producer', 'creator', 'creationdate', 'author', 'keywords', 'moddate', 'ptex.fullbanner', 'subject', 'title', 'trapped', 'source', 'total_pages', 'page', 'page_label']

[metadata examples]
producer        : pdfTeX-1.40.21
creator         : LaTeX with hyperref
creationdate    : 2021-06-22T01:27:10+00:00
author          : 
keywords        : 
moddate         : 2021-06-22T01:27:10+00:00
ptex.fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
subject         : 
title           : 
trapped         : /False
source          : https://arxiv.org/pdf/2103.15348.pdf
total_pages     : 16
page            : 0
page_label      : 1


## 5. PyMuPDFLoader <a id='pymupdf'></a>

`PyMuPDFLoader`는 속도 최적화가 되어 있으며, PDF 및 해당 페이지에 대한 자세한 메타데이터를 포함합니다.

### 설치
```bash
pip install -qU pymupdf
```

### 특징
- **빠른 처리 속도**: 대용량 PDF 처리에 적합
- **상세한 메타데이터**: 문서 구조 정보 포함
- **페이지당 하나의 문서**: 각 페이지를 별도 Document로 반환
- **텍스트 서식 보존**: 줄바꿈 등의 서식을 더 잘 보존

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
pymupdf_loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
pymupdf_docs = pymupdf_loader.load()

print(f"로드된 문서 수: {len(pymupdf_docs)}")
print(f"\n첫 번째 페이지 내용 (처음 400자):")
print(pymupdf_docs[0].page_content[:400])
print("\n줄바꿈이 잘 보존되어 있음을 확인할 수 있습니다.")

로드된 문서 수: 1

첫 번째 페이지 내용 (처음 400자):
<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
                                          2) 새로운 가치 창출
                                          3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음

줄바꿈이 잘 보존되어 있음을 확인할 수 있습니다.


In [9]:
# PyMuPDF의 메타데이터 확인
show_metadata(pymupdf_docs)
print("\nPyMuPDF는 더 많은 메타데이터 필드를 제공합니다 (예: format, file_path).")

[metadata keys]
['producer', 'creator', 'creationdate', 'source', 'file_path', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'moddate', 'trapped', 'modDate', 'creationDate', 'page']

[metadata examples]
producer     : PyPDF2
creator      : 
creationdate : 
source       : ../ADsP.pdf
file_path    : ../ADsP.pdf
total_pages  : 1
format       : PDF 1.4
title        : 
author       : 
subject      : 
keywords     : 
moddate      : 
trapped      : 
modDate      : 
creationDate : 
page         : 0

PyMuPDF는 더 많은 메타데이터 필드를 제공합니다 (예: format, file_path).


## 6. UnstructuredPDFLoader <a id='unstructured'></a>

[Unstructured](https://unstructured-io.github.io/unstructured/)는 비구조화된 또는 반구조화된 파일 형식을 다루기 위한 고급 도구입니다.

### 설치
```bash
pip install -qU unstructured
```

### 특징
- **고급 레이아웃 분석**: 문서 구조를 이해하고 요소별로 분류
- **다양한 모드 지원**:
  - `single`: 전체 문서를 하나로
  - `elements`: 요소별로 분할 (제목, 단락, 표 등)
- **카테고리 분류**: 각 요소를 Title, NarrativeText 등으로 분류

In [10]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# UnstructuredPDFLoader 인스턴스 생성 (기본 모드)
unstructured_loader = UnstructuredPDFLoader(FILE_PATH)

# 데이터 로드
unstructured_docs = unstructured_loader.load()

print(f"기본 모드 - 로드된 문서 수: {len(unstructured_docs)}")
print("첫 번째 문서 내용 (처음 300자):")
print(unstructured_docs[0].page_content[:300])

기본 모드 - 로드된 문서 수: 1
첫 번째 문서 내용 (처음 300자):
<ADsP 요약정리 및 오답노트>

-1과목-

(객관식) 데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성 Cinematch -> 넷플릭스에서 개발한 알고리즘 데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임 트레이딩, 공급, 수요예측 -> 에너지 산업 CRM -> 고객관계관리 데이터베이스 (기업내부) ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적 빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발

2) 새로운 가치 창출

3) 분석 기술 발전


In [11]:
# 기본 모드의 메타데이터 확인
show_metadata(unstructured_docs)

[metadata keys]
['source']

[metadata examples]
source : ../ADsP.pdf


In [12]:
# Elements 모드로 로더 생성
elements_loader = UnstructuredPDFLoader(FILE_PATH, mode="elements")

# 데이터 로드
elements_docs = elements_loader.load()

print(f"Elements 모드 - 로드된 문서 수: {len(elements_docs)}")
print()

# 처음 몇 개 요소 확인
for i, doc in enumerate(elements_docs[:2]):
    print(f"{['첫', '두'][i]} 번째 요소:")
    print(f"내용: {doc.page_content}")
    print(f"카테고리: {doc.metadata.get('category', 'N/A')}")
    print()

# 모든 카테고리 추출
categories = set(doc.metadata.get("category", "Unknown") for doc in elements_docs)
print(f"발견된 모든 카테고리: {categories}")

Elements 모드 - 로드된 문서 수: 3

첫 번째 요소:
내용: <ADsP 요약정리 및 오답노트>
카테고리: Title

두 번째 요소:
내용: -1과목-
카테고리: Title

발견된 모든 카테고리: {'Title', 'NarrativeText'}


In [13]:
# Elements 모드의 상세 메타데이터 확인
print("Elements 모드의 메타데이터:")
show_metadata(elements_docs)
print("\nElements 모드는 훨씬 더 풍부한 메타데이터를 제공합니다!")

Elements 모드의 메타데이터:
[metadata keys]
['source', 'coordinates', 'filename', 'languages', 'last_modified', 'filetype', 'category', 'element_id']

[metadata examples]
source        : ../ADsP.pdf
coordinates   : {'points': ((174.0, 203.0), (174.0, 230.0), (469.0, 230.0), (469.0, 203.0)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}
filename      : ADsP.pdf
languages     : ['kor']
last_modified : 2025-01-15T14:12:45
filetype      : application/pdf
category      : Title
element_id    : c3f96de214ab2a87f4a30c03f2c7cf35

Elements 모드는 훨씬 더 풍부한 메타데이터를 제공합니다!


## 7. PyPDFium2Loader <a id='pypdfium2'></a>

`PyPDFium2Loader`는 PDFium 라이브러리를 기반으로 한 로더로, Google Chrome의 PDF 렌더링 엔진을 사용합니다.

### 설치
```bash
pip install pypdfium2
```

### 특징
- **안정적인 렌더링**: Chrome의 PDF 엔진 사용
- **좋은 성능**: 빠른 처리 속도
- **깔끔한 텍스트 추출**: 레이아웃을 잘 보존

In [14]:
from langchain_community.document_loaders import PyPDFium2Loader

# PyPDFium2 로더 인스턴스 생성
pdfium2_loader = PyPDFium2Loader(FILE_PATH)

# 데이터 로드
pdfium2_docs = pdfium2_loader.load()

print(f"로드된 문서 수: {len(pdfium2_docs)}")
print(f"\n첫 번째 페이지 내용 (처음 400자):")
print(pdfium2_docs[0].page_content[:400])

로드된 문서 수: 1

첫 번째 페이지 내용 (처음 400자):
<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
 2) 새로운 가치 창출
3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음
사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에


In [15]:
# PyPDFium2의 메타데이터 확인
show_metadata(pdfium2_docs)

[metadata keys]
['title', 'author', 'subject', 'keywords', 'creator', 'producer', 'creationdate', 'moddate', 'source', 'total_pages', 'page']

[metadata examples]
title        : 
author       : 
subject      : 
keywords     : 
creator      : 
producer     : PyPDF2
creationdate : 
moddate      : 
source       : ../ADsP.pdf
total_pages  : 1
page         : 0


## 8. 로더 비교 및 선택 가이드 <a id='comparison'></a>

각 PDF 로더의 특징을 비교하고 상황별 추천 로더를 제시합니다.

In [16]:
# 로더 비교 분석
print("=== PDF 로더 비교 분석 ===")
print()

loaders_comparison = [
    ("PyPDFLoader", docs, "기본적이지만 빠른 텍스트 추출"),
    ("PyMuPDFLoader", pymupdf_docs, "빠르고 서식 보존 우수"),
    ("UnstructuredPDFLoader (기본)", unstructured_docs, "전체 문서를 하나로"),
    ("UnstructuredPDFLoader (elements)", elements_docs, "요소별 분류 및 분석"),
    ("PyPDFium2Loader", pdfium2_docs, "Chrome PDF 엔진 사용")
]

for name, loader_docs, desc in loaders_comparison:
    print(f"{loaders_comparison.index((name, loader_docs, desc)) + 1}. {name}")
    print(f"   - 문서 수: {len(loader_docs)}")
    print(f"   - 메타데이터 필드: {len(loader_docs[0].metadata.keys())}개")
    if name == "UnstructuredPDFLoader (elements)":
        categories = set(doc.metadata.get("category", "Unknown") for doc in loader_docs)
        print(f"   - 카테고리: {', '.join(categories)}")
    else:
        print(f"   - 첫 50자: {loader_docs[0].page_content[:50]}")
    print()

=== PDF 로더 비교 분석 ===

1. PyPDFLoader
   - 문서 수: 1
   - 메타데이터 필드: 7개
   - 첫 50자: <ADsP 요약정리 및 오답노트>-1과목-(객관식)데이터 마스킹 : 데이터의 속성은 유지한 채, 

2. PyMuPDFLoader
   - 문서 수: 1
   - 메타데이터 필드: 16개
   - 첫 50자: <ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성

3. UnstructuredPDFLoader (기본)
   - 문서 수: 1
   - 메타데이터 필드: 1개
   - 첫 50자: <ADsP 요약정리 및 오답노트>

-1과목-

(객관식) 데이터 마스킹 : 데이

4. UnstructuredPDFLoader (elements)
   - 문서 수: 3
   - 메타데이터 필드: 8개
   - 카테고리: Title, NarrativeText

5. PyPDFium2Loader
   - 문서 수: 1
   - 메타데이터 필드: 11개
   - 첫 50자: <ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성


### 상황별 추천 로더

| 상황 | 추천 로더 | 이유 |
|------|-----------|------|
| **빠른 처리가 필요한 경우** | PyMuPDFLoader | 가장 빠른 처리 속도, 안정적 |
| **간단한 텍스트 추출** | PyPDFLoader | 단순하고 가벼움 |
| **OCR이 필요한 경우** | PyPDFLoader + OCR | extract_images=True 옵션 사용 |
| **문서 구조 분석** | UnstructuredPDFLoader (elements) | 제목, 단락 등 요소별 분류 |
| **상세 메타데이터 필요** | PyMuPDFLoader 또는 UnstructuredPDFLoader | 풍부한 메타데이터 제공 |
| **안정적인 렌더링** | PyPDFium2Loader | Chrome의 PDF 엔진 사용 |

## 9. 핵심 요약 <a id='summary'></a>

### 🎯 주요 학습 내용

1. **PyPDFLoader**
   - 가장 기본적인 PDF 로더
   - OCR 지원 (rapidocr-onnxruntime 필요)
   - 간단한 사용 사례에 적합

2. **PyMuPDFLoader**
   - 빠른 처리 속도
   - 텍스트 서식 보존 우수
   - 대용량 PDF 처리에 적합

3. **UnstructuredPDFLoader**
   - 고급 문서 구조 분석
   - 요소별 분류 (제목, 단락 등)
   - 복잡한 문서 처리에 적합

4. **PyPDFium2Loader**
   - Chrome PDF 엔진 사용
   - 안정적인 렌더링
   - 균형잡힌 성능

### 💡 실무 팁

1. **로더 선택 기준**
   - 처리 속도 vs 기능의 균형 고려
   - 문서 유형에 따른 적절한 로더 선택
   - 필요한 메타데이터 수준 고려

2. **성능 최적화**
   - 대량 처리시 PyMuPDFLoader 사용
   - 메모리 사용량 모니터링
   - 필요시 배치 처리 구현

3. **문제 해결**
   - 깨진 텍스트: OCR 사용 고려
   - 복잡한 레이아웃: UnstructuredPDFLoader 사용
   - 메타데이터 부족: 다른 로더 시도

### 🔧 코드 예제

```python
# 상황별 로더 선택 예제
def select_pdf_loader(file_path, needs_ocr=False, needs_structure=False):
    if needs_structure:
        from langchain_community.document_loaders import UnstructuredPDFLoader
        return UnstructuredPDFLoader(file_path, mode="elements")
    elif needs_ocr:
        from langchain_community.document_loaders import PyPDFLoader
        return PyPDFLoader(file_path, extract_images=True)
    else:
        from langchain_community.document_loaders import PyMuPDFLoader
        return PyMuPDFLoader(file_path)
```